In [3]:
import torch
import numpy as np
from matplotlib import pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [8]:
m=1000
n=2
true_w=[2,-3,4]
true_b=4.2
features=torch.from_numpy(np.random.normal(0,1,(m,n)))
labels=true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels+=torch.from_numpy(np.random.normal(0,0.01,size=labels.size()))

In [10]:
plt.scatter(features[:,1],labels)

In [13]:
def data_iter(batch_size,features,labels):
    num_examples=len(features)
    indices=list(range(num_examples))
    np.random.shuffle(indices)
    for i in range(0,num_examples,batch_size):
        j=torch.LongTensor(indices[i:min(i+batch_size,num_examples)])
        yield features.index_select(0,j),labels.index_select(0,j)

In [14]:
batch_size=10
for X,y in data_iter(batch_size,features,labels):
    print(X,y)
    break

tensor([[-1.2074,  0.7194],
        [ 0.5890, -1.6221],
        [ 1.3483,  1.3167],
        [ 0.9048, -0.2384],
        [-1.4152, -1.4273],
        [-1.6490, -0.4000],
        [-0.1788,  0.3474],
        [-0.6418,  0.0103],
        [-0.0120,  0.1955],
        [-0.3107, -0.2873]], dtype=torch.float64) tensor([-0.3790, 10.2457,  2.9532,  6.7210,  5.6695,  2.1117,  2.7771,  2.8947,
         3.6053,  4.4461], dtype=torch.float64)


In [30]:
w=torch.tensor(np.random.normal(0,0.01,(n,1)),dtype=torch.float64)
b=torch.zeros(1,dtype=torch.float64)

In [31]:
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor([0.], dtype=torch.float64, requires_grad=True)

In [32]:
def linreg(X,w,b):
    return torch.mm(X,w)+b

In [33]:
def squared_loss(y_hat,y):
    return (y_hat-y.view(y_hat.size()))**2/2

In [34]:
def sgd(params,lr,batch_size):
    for p in params:
        p.data -= lr*p.grad /batch_size

In [35]:
lr=0.03
num_epochs=3
loss=squared_loss
net=linreg

for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        l=loss(net(X,w,b),y).sum()
        l.backward()
        sgd([w,b],lr,batch_size)
        
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l=loss(net(features,w,b),labels)
    print('epoch %d,loss: %f' % (epoch+1,train_l.mean().item()))
    

epoch 1,loss: 0.043508
epoch 2,loss: 0.000191
epoch 3,loss: 0.000050


torch.float32